### *3. Real data (9 points)*
#### In this assignment, you will estimate a hedonic pricing model using data on apartment prices in Poland. A hedonic pricing model estimates the value of a good based on its features. For apartments, the price depends on attributes such as area, number of rooms, distance to points of interests, etc.

In [2]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels.api as sm


Dataset Description
- **price**: Apartment price in PLN (Polish złoty).
- **month**: Month of year.
- **id**: Unique identifier for each listing.
- **type**: Type of apartment (e.g., flat, studio, etc.).
- **area**: Total usable area of the apartment (in m²).
- **rooms**: Number of rooms.
- **schoolDistance**: Distance to the nearest school (in km).
- **clinicDistance**: Distance to the nearest clinic or hospital (in km).
- **postOfficeDistance**: Distance to the nearest post office (in km).
- **kindergartenDistance**: Distance to the nearest kindergarten (in km).
- **restaurantDistance**: Distance to the nearest restaurant (in km).
- **collegeDistance**: Distance to the nearest college/university (in km).
- **pharmacyDistance**: Distance to the nearest pharmacy (in km).
- **ownership**: Type of ownership (e.g., freehold, cooperative).
- **buildingMaterial**: Main material used for construction (e.g., brick, concrete).
- **hasParkingSpace**: Boolean (1/0) indicating if a parking space is available.
- **hasBalcony**: Boolean (1/0) indicating if the apartment has a balcony.
- **hasElevator**: Boolean (1/0) indicating if the building has an elevator.
- **hasSecurity**: Boolean (1/0) indicating if the building has security features.
- **hasStorageRoom**: Boolean (1/0) indicating if the apartment has a storage room.


In [3]:
df = pd.read_csv('../Input/apartments.csv')
df.head(5)

,id,price,month,area,type,rooms,centredistance,schooldistance,clinicdistance,postofficedistance,...,restaurantdistance,collegedistance,pharmacydistance,ownership,buildingmaterial,hasparkingspace,hasbalcony,haselevator,hassecurity,hasstorageroom
0,a01d82c9529f98a54d64b9e061c9a73b,1199999,1,105,apartmentBuilding,4,5.06,1.080,0.949,0.623,...,1.054,3.062,0.335,condominium,brick,no,yes,yes,no,no
1,8373aa373dbc3fe7ca3b7434166b8766,650000,1,73,tenement,3,3.24,0.275,0.672,0.367,...,0.300,1.857,0.280,condominium,brick,no,no,no,no,no
2,7d0c31d5409caab173571cce3dcdf702,590000,1,69,blockOfFlats,3,3.94,0.139,1.336,0.926,...,0.071,0.786,0.304,condominium,brick,no,yes,no,no,yes
3,3eaa36a59b9354206703b5f6b2f2ff1d,584999,1,42,blockOfFlats,2,5.19,0.209,1.533,0.201,...,0.157,2.722,0.257,condominium,NaN,no,yes,no,no,no
4,027b30cebbc49faf3094421b741ddd56,363000,1,46,blockOfFlats,2,1.65,0.250,0.624,0.510,...,0.232,0.986,0.273,condominium,concreteSlab,no,no,no,no,yes


In [4]:
df.count()

id                      110191
price                   110191
month                   110191
area                    110191
type                     86863
rooms                   110191
centredistance          110191
schooldistance          110115
clinicdistance          109844
postofficedistance      110068
kindergartendistance    110077
restaurantdistance      109958
collegedistance         107260
pharmacydistance        110054
ownership               110191
buildingmaterial         65926
hasparkingspace         110191
hasbalcony              110191
haselevator             104858
hassecurity             110191
hasstorageroom          110191
dtype: int64

In [5]:
df.isnull().sum()

id                          0
price                       0
month                       0
area                        0
type                    23328
rooms                       0
centredistance              0
schooldistance             76
clinicdistance            347
postofficedistance        123
kindergartendistance      114
restaurantdistance        233
collegedistance          2931
pharmacydistance          137
ownership                   0
buildingmaterial        44265
hasparkingspace             0
hasbalcony                  0
haselevator              5333
hassecurity                 0
hasstorageroom              0
dtype: int64

In [6]:
for c in ["type", "buildingmaterial"]:
    if c in df.columns:
        s = df[c].astype("category")
        if "Missing" not in s.cat.categories:
            s = s.cat.add_categories(["Missing"])
        df[c] = s.fillna("Missing")


In [7]:
df.count()

id                      110191
price                   110191
month                   110191
area                    110191
type                    110191
rooms                   110191
centredistance          110191
schooldistance          110115
clinicdistance          109844
postofficedistance      110068
kindergartendistance    110077
restaurantdistance      109958
collegedistance         107260
pharmacydistance        110054
ownership               110191
buildingmaterial        110191
hasparkingspace         110191
hasbalcony              110191
haselevator             104858
hassecurity             110191
hasstorageroom          110191
dtype: int64

In [8]:
df.dropna(inplace=True)
df.count()

id                      101715
price                   101715
month                   101715
area                    101715
type                    101715
rooms                   101715
centredistance          101715
schooldistance          101715
clinicdistance          101715
postofficedistance      101715
kindergartendistance    101715
restaurantdistance      101715
collegedistance         101715
pharmacydistance        101715
ownership               101715
buildingmaterial        101715
hasparkingspace         101715
hasbalcony              101715
haselevator             101715
hassecurity             101715
hasstorageroom          101715
dtype: int64

#### 3a Cleaning (2 points)

In [9]:
df["area2"] = df["area"] ** 2

In [10]:
cols = ['hasparkingspace', 'hasbalcony', 'haselevator', 'hassecurity', 'hasstorageroom']
df[cols] = df[cols].replace({'yes': 1, 'no': 0})
df.loc[:, "hasparkingspace":"hasstorageroom"].head(5)


C:\Users\diego\AppData\Local\Temp\ipykernel_28328\1462104389.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[cols] = df[cols].replace({'yes': 1, 'no': 0})


,hasparkingspace,hasbalcony,haselevator,hassecurity,hasstorageroom
0,0,1,1,0,0
1,0,0,0,0,0
2,0,1,0,0,1
3,0,1,0,0,0
4,0,0,0,0,1


In [11]:
# Obtener el último dígito de 'area'
df["last_digit"] = df["area"] % 10

# Crear las variables dummy end_0 ... end_9
for i in range(10):
    df[f"end_{i}"] = (df["last_digit"] == i).astype(int)

# Verificamos las primeras filas
df.head(10)[["area", "last_digit"] + [f"end_{i}" for i in range(10)]]


,area,last_digit,end_0,end_1,end_2,end_3,end_4,end_5,end_6,end_7,end_8,end_9
0,105,5,0,0,0,0,0,1,0,0,0,0
1,73,3,0,0,0,1,0,0,0,0,0,0
2,69,9,0,0,0,0,0,0,0,0,0,1
3,42,2,0,0,1,0,0,0,0,0,0,0
4,46,6,0,0,0,0,0,0,1,0,0,0
5,81,1,0,1,0,0,0,0,0,0,0,0
6,70,0,1,0,0,0,0,0,0,0,0,0
7,67,7,0,0,0,0,0,0,0,1,0,0
8,69,9,0,0,0,0,0,0,0,0,0,1
9,56,6,0,0,0,0,0,0,1,0,0,0


#### 3b Linear model estimation (4 points)

In [23]:
# Columnas numéricas (asegurar tipo)
num_cols = [
    "area","area2",
    "schooldistance","clinicdistance","postofficedistance",
    "kindergartendistance","restaurantdistance","collegedistance","pharmacydistance",
    "hasparkingspace","hasbalcony","haselevator","hassecurity","hasstorageroom"
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Dummies (base = 9)
end_dum = df[[f"end_{i}" for i in range(10) if f"end_{i}" in df.columns]].copy()
end_dum = end_dum.drop(columns=["end_9"], errors="ignore")     # omite la base
end_dum = end_dum.apply(pd.to_numeric, errors="coerce")        # por si quedaron en object

# Categóricas ->
cat_cols = ["month","type","rooms","ownership","buildingmaterial"]
cats_present = [c for c in cat_cols if c in df.columns]
cat_dum = pd.get_dummies(df[cats_present], drop_first=True, dtype=float)

# Matriz X e y 
parts = [df[[c for c in num_cols if c in df.columns]], end_dum, cat_dum]
X = pd.concat(parts, axis=1)
y = pd.to_numeric(df["price"], errors="coerce")

# Ajuste del modelo
mask = X.notna().all(axis=1) & y.notna()
X_ = sm.add_constant(X.loc[mask].astype(float), has_constant="add")
y_ = y.loc[mask].astype(float)

model = sm.OLS(y_, X_).fit()      
print(model.summary())

# Comentario sobre end_0 (vs base end_9):
coef = model.params["end_0"]
pval = model.pvalues["end_0"]
print(f"\nend_0 → coef = {coef} | p-valor = {pval}")




                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.570
Method:                 Least Squares   F-statistic:                     4215.
Date:                Wed, 03 Sep 2025   Prob (F-statistic):               0.00
Time:                        10:57:01   Log-Likelihood:            -1.4226e+06
No. Observations:              101715   AIC:                         2.845e+06
Df Residuals:                  101682   BIC:                         2.846e+06
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

In [ ]:
 #Perform the same regression but this time by partialling-out. 
# Your target parameter will be the one associated with end_0. 
# Print a summary table and verify the coefficients are the same with both methods.

# 1) OLS completo
full = sm.OLS(y, sm.add_constant(X)).fit()

# 2) Partialling-out (FWL) manual
d = X["end_0"] # variable endógena
Z = X.drop(columns=["end_0"]) # resto de covariables
y_res = sm.OLS(y, sm.add_constant(Z)).fit().resid
d_res = sm.OLS(d, sm.add_constant(Z)).fit().resid
po = sm.OLS(y_res, d_res).fit()   # regresión de residuos (sin constante)

# 3) Resumen mini y verificación
tabla = pd.DataFrame({
    "method": ["Full OLS", "Partialling-out"],
    "coef_end_0": [full.params["end_0"], po.params[0]],
    "p_value":    [full.pvalues["end_0"], po.pvalues[0]]
})
print(tabla)
print("¿Coeficientes iguales? ", np.allclose(tabla.loc[0,"coef_end_0"], tabla.loc[1,"coef_end_0"]))


            method    coef_end_0   p_value
0         Full OLS  18329.817439  0.000008
1  Partialling-out  18329.817439  0.000008
¿Coeficientes iguales?  True


C:\Users\diego\AppData\Local\Temp\ipykernel_28328\3946850938.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "coef_end_0": [full.params["end_0"], po.params[0]],
C:\Users\diego\AppData\Local\Temp\ipykernel_28328\3946850938.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "p_value":    [full.pvalues["end_0"], po.pvalues[0]]


#### 3c  Price premium for area that ends in 0-digit (3 points)

In [17]:
idx = X.index                               # mismas filas que usaste en X, y
train_mask = df.loc[idx, "last_digit"] != 0 # Separa los que terminan en 0

X_train = X.loc[train_mask].drop(columns=["end_0"], errors="ignore")
y_train = y.loc[train_mask]

model_no0 = sm.OLS(y_train, sm.add_constant(X_train, has_constant="add")).fit()
print(model_no0.summary())   

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.569
Model:                            OLS   Adj. R-squared:                  0.569
Method:                 Least Squares   F-statistic:                     3840.
Date:                Wed, 03 Sep 2025   Prob (F-statistic):               0.00
Time:                        10:42:46   Log-Likelihood:            -1.2596e+06
No. Observations:               90116   AIC:                         2.519e+06
Df Residuals:                   90084   BIC:                         2.520e+06
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

In [18]:

cols = [c for c in model_no0.model.exog_names if c != "const"]
X_pred = X.reindex(columns=cols, fill_value=0)
X_pred = sm.add_constant(X_pred, has_constant="add")

df.loc[idx, "price_pred_no0model"] = model_no0.predict(X_pred)

In [20]:
mask_end0 = df.loc[idx, "last_digit"] == 0

avg_actual = df.loc[mask_end0, "price"].mean()
avg_pred   = df.loc[mask_end0, "price_pred_no0model"].mean()
premium    = avg_actual - avg_pred

print(f"Average ACTUAL (end_0): {avg_actual:,.2f}")
print(f"Average PREDICTED:      {avg_pred:,.2f}")
print(f"Actual - Predicted:     {premium:,.2f}")

Average ACTUAL (end_0): 881,339.19
Average PREDICTED:      863,126.83
Actual - Predicted:     18,212.36


Para los casa con área que termina en 0, el precio real promedio supera al predicho en 18,212, es decir, 2.1% por encima, esto sugiere un pequeño “premium” para áreas que acaban en 0.